In [1]:
from keras.layers import Lambda, Input, Dense
from keras.models import Models
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

Using TensorFlow backend.


ImportError: cannot import name 'Models'

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
clip_scans_at = 8
data = np.loadtxt("../../dataset/diag_labrococo.txt").astype('float32')
np.clip(data, a_min=0, a_max=clip_scans_at, out=data);

In [ ]:
ts = data[:, :7]
# scans = data[:, 7:] / clip_scans_at
scans = data[:, 100:600] / clip_scans_at  # robot embodiment

print(scans.shape, np.max(scans), np.min(scans))

In [ ]:
scan_idx = 50
plt.figure(figsize=(10, 2))
#for i in range(int(scans.shape[0] / 2)):
plt.plot(np.arange(scans.shape[1]), scans[scan_idx,:])

In [ ]:
split_data_at = .9
x_train = scans[:int(scans.shape[0]*split_data_at),:]
x_test = scans[int(scans.shape[0]*split_data_at):,:]
original_dim = scans.shape[1]

print("scans:", scans.shape)
print("xtrain:", x_train.shape)
print("xtest:", x_test.shape)

In [ ]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 128
batch_size = 128
latent_dim = 10

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
#plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
#plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

In [ ]:
# models = (encoder, decoder)
# data = x_test
reconstruction_loss = binary_crossentropy(inputs, outputs)

In [ ]:
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()
# plot_model(vae, to_file='vae_mlp.png', show_shapes=True)

In [ ]:
epochs = 100

vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose = 2,
        validation_data=(x_test, None))

In [ ]:
# plot
# display a 2D plot of the digit classes in the latent space
z_mean, _, _ = encoder.predict(x_test, batch_size=batch_size)
print(z_mean.shape)
x_decoded = decoder.predict(z_mean)
print(x_decoded.shape)

In [ ]:
scan_idx = 50
plt.figure(figsize=(10, 2))
plt.plot(np.arange(x_test.shape[1]), x_test[scan_idx,:])
plt.plot(np.arange(x_test.shape[1]), x_decoded[scan_idx,:])